# Perpetual Bermudan Options

Click here for an interactive version of this notebook:-
[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/pjohno/MSc-Math-Finance-2018/master?filepath=notebooks%2FMSc%20Project%207%20-%20Perpetual%20Bermudans.ipynb)


Consider the well-known Black and Scholes (1973) partial differential equation for an option with an underlying
asset following geometric Brownian motion:
$$
\frac{\partial{V}}{\partial{t}} + \frac{1}{2} {\sigma}^2 S^2
\frac{\partial {^2V}}{\partial{S^2}} + (r-D_c) S
\frac{\partial{V}}{\partial{S}} - rV = 0,
$$
where $V(S,t)$ is the price of the derivative product, $S$ the
current value of the underlying asset, $t$ is time,  $T$ is the time to maturity
 $r$ the risk-free interest rate, $\sigma$ the
volatility of the underlying asset and $X$ is the exercise price of the option. 
$D_c$ is a continuous dividend yield which could, for example, be the foreign 
interest rate in a foreign exchange option.


Assume now that the option is a Bermudan option that can be exercised at fixed interval periods such that $t=t^k = T - k\Delta T$ for integer values of $k$, and where $T$ is the maturity date of the option. 

Let $V(S,t;t^k)$ be the value of the option at time $t$, in which the next bermudan exercise opportunity is at $t^k$.
Then for a Bermudan put option,  we can find the value of the option $V(S,t;t^k)$ for $t^{k+1}\leq t <t^k$ by solving the BS PDE with final condition given as

$$
\quad V(S,T - k\Delta T;T - k\Delta T) = \left\{ 
\begin{array}{cc}
X - S & \text{ if } S < \theta^k  \\
V(S,T -k\Delta T ;T - (k-1)\Delta T ) & \text{ if } S>\theta^k
\end{array}
\right.
$$

where $\theta^k$ is the optimal exercise decision point at $t^k$, at which
$$
S - \theta^k = V(\theta^k,t^k ; t^{k-1} )
$$
holds true. 

If the final maturity date of the option is far into the future and $T\rightarrow\infty$, then we should find that
$$
\lim_{k\rightarrow\infty} \theta^k = \bar \theta
$$
for some constant $\bar \theta$ and 
$$
\lim_{k\rightarrow\infty} \bigg[ V(S,t ; t^{k+1} ) - V(S,t + \Delta T ; t^k ) \bigg] = 0 \quad \forall S,t .
$$

Next make the following standard
transformations                
$$
x = \log(S_0),
$$
$$
y = \log(S_{T}).
$$
The value of 
the option at time $t=t^{k+1}$ on an underlying asset  $S_0$ where $S_0>\theta^{k+1}$, has an  exact solution given by 

$$
V(x,t^{k+1};t^k) = A (x) \int_{-\infty}^{\log(\theta^k)} B(x,y) (X - e^y)\, dy + A (x) \int_{\log(\theta^k)}^{\infty} B(x,y)V(y,t^{k};t^{k-1}) \, dy,
$$
where, 
$$
A(x) = \frac{1}{\sqrt{2\sigma^2\pi \Delta T}} e^{-\frac12 kx-\frac18 
\sigma^2k^2 \Delta T - r \Delta T},
$$
and
$$
B(x,y) = e^{-\frac{(x-y)^2}{2\sigma^2 \Delta T} + \frac12 ky}
$$
and
$$
k = \frac{2(r-D_c)}{{\sigma}^2} -1.
$$

For the perpetual option, we solve for $k$ increasing until the difference between successive values of $\theta^k$ are sufficiently small. 

We solve this problem by setting up a structure containing all the relavent functions. This will contain the functions $A$, $B$, a method for integrating and a method to return the value $V(x,0)$. First include libraries:

In [1]:
#include <iostream>
#include <algorithm>
#include <cmath>
#include <vector>
#include <functional>

Now assume that the value of the Bermudan option $V(y,t^k;t^{k-1})$ can be approximated by discrete set of points on a fixed grid. First assume that $\theta^0$ is given, so that we can create a grid
$$
y_i = log(\theta^0) + i dy \quad \text{ for } i = 0,1,2,\dots N
$$
where $y_N$ is sufficiently large.

The we can say that 
$$
V(y_i,t^k;t^{k-1}) = v_i^k.
$$
An appropriate starting point for this calculation would be to set $\theta^0$ and $v_i^k$ to the appropriate perpetual put option values. Then we given a guess can solve for $V$ at $t^{k+1}$ using the formula
$$
V(x,t^{k+1};t^{k}) = \Phi(x,\theta^0) + A(x) \sum_{i=0}^N B(x,y_i) w_i v^k_i 
$$
where $\Phi$ represents an adapted cummulative normal distribution to solve the left integral, and $w_i$ is the weighting for an appropriate numerical integration.

First create functions $A$ and $B$.

In [2]:
// the function A
double A(double x, double r, double sigma, double k, double dt)
{
    return 1. / sqrt(2 * sigma*sigma*M_PI*dt) * exp(-0.5*k*x - 0.125*sigma*sigma*k*k*dt - r*dt);
}

In [3]:
// the function B
double B(double x, double y, double sigma, double k, double dt)
{
    return exp(-(x - y)*(x - y) / (2.*sigma*sigma*dt) + 0.5 * k * y);
}

Next create an appropriate intregration algorithm


In [4]:
// integration proceedure: integrate the function f_i at the points y_i, contained in a vector integrand
// PLEASE note that here we are assuming that the grid is equally spaced
// nDown, nUp reference the subsection of the vector y we wish to integrate over
double integrate(int nDown, int nUp,const std::vector<double> &y,const std::vector<double> &integrand)
{
    double sum, h = (y[nUp] - y[nDown]) / (nUp - nDown);
    sum = integrand[nDown];
    for (int i = nDown+2; i<nUp; i+=2)
    {
        sum += 2.*integrand[i];
    }
    for (int i = nDown+1; i<nUp; i+=2)
    {
        sum +=  4.*integrand[i];
    }
    sum += integrand[nUp];
    return h / 3.*sum;
}

Now we can value the payoff from an option by inputing the grid, and the value function in that region.

In [5]:
double valueOption(double  x,double r,double sigma,double T,int n,
                   const std::vector<double> &y,const std::vector<double> &payoff_vec)
{
    // variable k
    double k = 2.*r / sigma / sigma - 1.;
           
    // setup vectors to store option values, and integrand function
    // these vectors must be the same size as the y vector
    std::vector<double> integrand(y.size());
    
    for (uint j = 0; j < y.size(); j++)
    {
        integrand[j] = B( x , y[j] , sigma , k , T ) * payoff_vec[j];
    }
    // in this example we integrate over the entire range of y, it is possible to select a subrange
    return A(x, r, sigma, k, T )*integrate(0, n, y, integrand);
      
}

For example, let us set up a simple scenario:

In [6]:
double  S,X,r,sigma,dT;

In [7]:
S = 75; X = 100.; r = 0.06; sigma = 0.2; dT = 0.1;

{
    int N=500;
    double x0 = log(S);
    
    std::cout << " A perpetual Bermudan put option ...\n";
    
    // for a perpetual American put option we have 
    double alpha = -2*r/sigma/sigma;
    double theta = X/(1-1./alpha);
    std::cout << " Initially, we take prepetual American boundary: theta^0 = " << theta << "\n";
    double A = -1./alpha*pow(theta,1-alpha);
    std::cout << " and: P^0(S) = " << A << " * S^{"<< alpha << "} \n\n";
    
    std::vector<double> yDown(N+1),VDown(N+1);
    { 
      double yMin = log(X*exp(-7.5*sigma*sqrt(dT)));
      double yMax = log(theta);
      double dy = (yMax - yMin)/N;
      // assign grid and corresponding payoff
      for(int i=0;i<=N;i++)
      {
            yDown[i] = yMin + i*dy;
            VDown[i] = X - exp(yDown[i]);
      }
    }
    
    // set y grid up at the final step, using theta^0 as lower limit
    std::vector<double> yUp(N+1),VUp(N+1);
    {
      double yMin = log(theta);
      double yMax = log(X*exp(7.5*sigma*sqrt(dT)));
      double dy = (yMax - yMin)/N;
      // assign grid and corresponding payoff
      for(int i=0;i<=N;i++)
      {
            yUp[i] = yMin + i*dy;        
            VUp[i] = A*exp(alpha*yUp[i]);
      }
    }
    
    std::cout << " Next calculate P^1(S) = A(x)*\\int^\\theta B(x,y)*(X-e^y) dy + A(x)*\\int_\\theta B(x,y)*V(y,T) dy \n\n";
    // return the value V(x=log(S),0) = A(x)*\int^\theta B(x,y)*(X-e^y) dy + A(x)*\int_\theta B(x,y)*V(y,T) dy
    double value = valueOption(x0,r,sigma,dT,N,yDown,VDown) + valueOption(x0,r,sigma,dT,N,yUp,VUp);
    // get equivalent perpetual value
    if(S<theta)
      std::cout << " P^1(S="<<S<<") = " << value << "  ::  P^0(S="<<S<<") = " << X - S << "\n";
    else
      std::cout << " P^1(S="<<S<<") = " << value << "  ::  P^0(S="<<S<<") = " << A*pow(S,alpha) << "\n";
    
    std::cout << " To find the value of theta^1, we require \n X - theta^1 - P^1(theta^1) = 0 \n In this case we have:- \n";
    std::cout << X << " -  " << S << " - P^1(S="<<S<<") = " << X - S - value << "\n";

}

 A perpetual Bermudan put option ...
 Initially, we take prepetual American boundary: theta^0 = 75
 and: P^0(S) = 1.05469e+07 * S^{-3} 

 Next calculate P^1(S) = A(x)*\int^\theta B(x,y)*(X-e^y) dy + A(x)*\int_\theta B(x,y)*V(y,T) dy 

 P^1(S=75) = 24.6612  ::  P^0(S=75) = 25
 To find the value of theta^1, we require 
 X - theta^1 - P^1(theta^1) = 0 
 In this case we have:- 
100 -  75 - P^1(S=75) = 0.338752


Can you find the value of $S$ which satisfies the equation for $\theta^1$?